In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### TO DO:
    z_score a gör outlier olanları veriden at
    veriyi tekrar returnu 1 olanlarla concat et
    veriden gereksiz kolonlar -id gibi- at
    veriyi normalize et
    veriyi shuffle et
    gerekirse veriyi yarıya indir modele öyle ver
    dt, rf, lightgbm, xgboost algolarını dene 

In [2]:
train = pd.read_csv("train_all.csv")
train

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,...,unresolved_accepted_claim,unresolvedclaim_percentage,defective_returnRate,review_rates_mean,review_rates_min,review_rates_max,brand_id,brand_name,category_id,category_name
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.90,51.94,...,25,0.328947,0.003045,3.250000,1,5,682,Slazenger,604,T-Shirt
1,2021-05-01 00:00:01.761000+00:00,3941815,0,157306,901280145,595758968,36888320,65612791,59.99,49.99,...,7,0.777778,0.000744,0.000000,0,0,381,Ltb,2949,Polo Yaka T-shirt
2,2021-05-01 00:00:01.761000+00:00,3941815,0,163856,901280144,595758968,36888399,65613254,49.90,49.90,...,3,0.200000,0.000633,0.000000,0,0,381,Ltb,2949,Polo Yaka T-shirt
3,2021-05-01 00:00:01.761000+00:00,3941815,0,163,901280149,595758968,36313798,64622938,129.95,59.97,...,0,0.000000,0.000545,4.476190,1,5,121,Pierre Cardin,604,T-Shirt
4,2021-05-01 00:00:02.742000+00:00,15996105,1,968,901279297,595758987,96930601,161408176,89.99,71.99,...,0,0.000000,0.002828,4.500000,2,5,40,TRENDYOLMİLLA,1182,Elbise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10702712,2021-07-30 23:59:58.335000+00:00,14877765,0,112044,1106635719,721367377,120102810,193463391,179.95,179.95,...,25,0.050302,0.003462,5.000000,5,5,4951,Pull & Bear,526,Etek
10702713,2021-07-30 23:59:59.290000+00:00,25067941,1,163,1106635937,721367381,123501317,199281489,199.95,71.98,...,0,0.000000,0.000545,4.480000,1,5,19,U.S. Polo Assn.,604,T-Shirt
10702714,2021-07-30 23:59:59.290000+00:00,25067941,1,163,1106635936,721367381,123501233,199281191,169.95,62.38,...,0,0.000000,0.000545,4.263158,1,5,19,U.S. Polo Assn.,604,T-Shirt
10702715,2021-07-30 23:59:59.290000+00:00,25067941,1,163,1106635935,721367381,94575220,157698762,349.95,143.98,...,0,0.000000,0.000545,5.000000,5,5,19,U.S. Polo Assn.,597,Gömlek


In [3]:
train_0 = train[train.is_returned == 0]
train_1 = train[train.is_returned == 1]

In [ ]:
train_0.isna().sum()

In [ ]:
train_0.info()

In [5]:
#numeric olan değerleri z_score a veriyorum
z_score = zscore(train_0[['original_price', 'discounted_price', 'coupon_discount', "returnRate",
   "total_claim", "unresolved_claim", "unresolved_accepted_claim", "unresolvedclaim_percentage",
    "defective_returnRate", "review_rates_mean", "review_rates_min", "review_rates_max"]])

In [ ]:
train_0.shape

In [6]:
abs_zscore = abs(z_score)
train_0 = train_0[(abs_zscore < 2).all(axis=1)]

In [ ]:
train_0.shape

In [ ]:
train_1.shape

In [7]:
train_zscore = pd.concat([train_0, train_1])

In [ ]:
train_zscore.columns

In [8]:
drop_cols = ["order_date", "user_id", "supplier_id", "order_line_item_id", "order_parent_id",
             "product_content_id", "product_variant_id", "coupon_id", "promotion_name",
            'is_wallet_trx', 'is_saved_card_trx', 'brand_id', 'brand_name', 'category_id', 'category_name',
            "review_rates_min", "review_rates_max"]

In [9]:
train = train_zscore.drop(drop_cols, axis=1)
train.head()

,is_elite_user,original_price,discounted_price,ship_cost,coupon_discount,promotion_award_value,is_returned,returnRate,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage,defective_returnRate,review_rates_mean
0,0,79.90,51.94,8.880000,0.0,35.0,0.0,0.098178,16357,76,25,0.328947,0.003045,3.250000
3,0,129.95,59.97,9.400000,0.0,40.0,0.0,0.050737,46699,0,0,0.000000,0.000545,4.476190
4,1,89.99,71.99,9.400000,0.0,20.0,0.0,0.127110,3269293,0,0,0.000000,0.002828,4.500000
5,1,56.99,28.50,1.342857,0.0,50.0,0.0,0.127110,3269293,0,0,0.000000,0.002828,4.416667
6,1,69.99,28.00,1.342857,0.0,60.0,0.0,0.127110,3269293,0,0,0.000000,0.002828,4.416667


In [10]:
train = train.fillna(0)

In [ ]:
train.isnull().sum()

In [11]:
df_sampled = train.sample(frac=0.5, random_state=42)

X = df_sampled.drop(columns="is_returned", axis=1)
y = df_sampled["is_returned"]

In [12]:
X = StandardScaler().fit(X).transform(X.astype(float))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [16]:
preds = rf.predict(X_test)

print(confusion_matrix(preds, y_test))
print(classification_report(preds, y_test))

[[667270  41619]
 [ 17191  94632]]
              precision    recall  f1-score   support

         0.0       0.97      0.94      0.96    708889
         1.0       0.69      0.85      0.76    111823

    accuracy                           0.93    820712
   macro avg       0.83      0.89      0.86    820712
weighted avg       0.94      0.93      0.93    820712



In [18]:
!pip install lightgbm

In [19]:
from lightgbm import LGBMClassifier, plot_importance

lgbm = LGBMClassifier(random_state=42)

In [20]:
%%time

lgbm.fit(X_train, y_train)

Wall time: 20.9 s


LGBMClassifier(random_state=42)

In [22]:
preds_lgbm = lgbm.predict(X_test)

print(confusion_matrix(preds_lgbm, y_test))
print(classification_report(preds_lgbm, y_test))

[[671573  49206]
 [ 12888  87045]]
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96    720779
         1.0       0.64      0.87      0.74     99933

    accuracy                           0.92    820712
   macro avg       0.81      0.90      0.85    820712
weighted avg       0.94      0.92      0.93    820712



In [23]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

In [24]:
%%time

dt.fit(X_train, y_train)

Wall time: 2min 19s


DecisionTreeClassifier(random_state=42)

In [25]:
preds_dt = dt.predict(X_test)

print(confusion_matrix(preds_dt, y_test))
print(classification_report(preds_dt, y_test))

[[663377  42954]
 [ 21084  93297]]
              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95    706331
         1.0       0.68      0.82      0.74    114381

    accuracy                           0.92    820712
   macro avg       0.83      0.88      0.85    820712
weighted avg       0.93      0.92      0.92    820712



In [26]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42)

In [27]:
%%time

xgb.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:00:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 4min 34s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
preds_xgb = xgb.predict(X_test)

print(confusion_matrix(preds_xgb, y_test))
print(classification_report(preds_xgb, y_test))

[[672567  47000]
 [ 11894  89251]]
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96    719567
         1.0       0.66      0.88      0.75    101145

    accuracy                           0.93    820712
   macro avg       0.82      0.91      0.85    820712
weighted avg       0.94      0.93      0.93    820712



In [ ]:
# TODO: feature_importance